<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/examples/retrievers/vectara_auto_retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto-Retrieval from a Vectara Index

This guide shows how to perform **auto-retrieval** in LlamaIndex with Vectara. 

Given a natural language query, we first use the LLM to infer a set of metadata filters as well as the right query string to pass to the Vectara Index.

This allows for more dynamic, expressive forms of retrieval beyond top-k semantic search. The relevant context for a given query may only require filtering on a metadata tag, or require a joint combination of filtering + semantic search within the filtered set, or just raw semantic search.

## Setup 

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [1]:
!pip install llama-index

In [2]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
from llama_index.schema import TextNode
from llama_index.indices.managed.types import ManagedIndexQueryMode
from llama_index.indices import VectaraIndex
from llama_index.indices.managed.vectara import VectaraAutoRetriever

from llama_index.vector_stores.types import MetadataInfo, VectorStoreInfo

from llama_index.llms import OpenAI
from llama_index import ServiceContext

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


## Defining Some Sample Data

We insert some sample nodes containing text chunks into Vectara. Note that each `TextNode` not only contains the text, but also certain metadata fields like year, director, rating or genre. 
In Vectara you will need to [define](https://docs.vectara.com/docs/learn/metadata-search-filtering/filter-overview) these metadata fields in your coprus.

In [4]:
nodes = [
    TextNode(
        text="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={ "year": 1993, "rating": 7.7, "genre": "science fiction" },
    ),
    TextNode(
        text="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    TextNode(
        text="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    TextNode(
        text="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    TextNode(
        text="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    TextNode(
        text="Barbie suffers a crisis that leads her to question her world and her existence.",
        metadata={"year": 2023, "director": "Greta Gerwig", "genre": "fantasi", "rating": 9.5},
    ),
]

## Build Vectara Managed Index

Now we load our sample data into the Vectara Index.

In [5]:
##index = VectaraIndex.from_documents(nodes)

In [6]:
index = VectaraIndex(nodes)

LLM is explicitly disabled. Using MockLLM.


## Setup OpenAI

Auto-retrieval uses an LLM to convert the natural language query into a shorter query and meta data filtering conditions. We will be using the OpenAI LLM, so let's set that up here.

In [7]:
import getpass
import openai

if not os.environ.get("OPENAI_API_KEY", None):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

openai.api_key = os.environ["OPENAI_API_KEY"]

## Define `VectorStoreInfo`

We define `VectorStoreInfo`, which contains a structured description of the metadata filters suported. This information is later on usedin the auto-retrieval prompt where the LLM infers metadata filters.

In [8]:

vector_store_info = VectorStoreInfo(
    content_info="information about a movie",
    metadata_info=[
        MetadataInfo(
            name="genre",
            description="The genre of the movie. One of ['science fiction', 'comedy', 'drama', 'thriller', 'romance', 'action', 'animated']",
            type="string",
        ),
        MetadataInfo(
            name="year",
            description="The year the movie was released",
            type="integer",
        ),
        MetadataInfo(
            name="director",
            description="The name of the movie director",
            type="string",
        ),
        MetadataInfo(
            name="rating", 
            description="A 1-10 rating for the movie", 
            type="float"
        ),
    ],
)

## Running over some sample data

Now let's create a `VectaraAutoRetriever` instance and run some example queries.


In [9]:
service_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-4-1106-preview", temperature=0)
)
retriever = VectaraAutoRetriever(
    index, vector_store_info=vector_store_info, service_context=service_context,
)

In [10]:
retriever.retrieve("movies directed by Greta Gerwig")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.indices.managed.vectara.retriever:Using query str: movies
Using query str: movies
INFO:llama_index.indices.managed.vectara.retriever:Using implicit filters: [('director', '==', 'Greta Gerwig')]
Using implicit filters: [('director', '==', 'Greta Gerwig')]


[NodeWithScore(node=TextNode(id_='19ce35022e05e1d8672d1c609ce71b7b0958e1ecfc61717f54d3df26bf43b2392dc12b4148842e582d50ba813aa9381141e9ddefd4f6315da36e85efb77844f5', embedding=None, metadata={'lang': 'eng', 'offset': '0', 'len': '79', 'year': '2023', 'director': 'Greta Gerwig', 'genre': 'fantasi', 'rating': '9.5'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='5d2839ded8712eeae0821b73d7eefedc5d3a3920bcd08073cddec1f13edfe429', text='Barbie suffers a crisis that leads her to question her world and her existence.', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.5710437),
 NodeWithScore(node=TextNode(id_='753a9f1e10994483af6409bd4651f8e2d17622216f410ed58630503e270f4b1531dc7822626c4055e99db8f30397e1c61545e988f6ff7333a8ec61509cdcd94d', embedding=None, metadata={'lang': 'eng', 'offset': '0', 'len': '82', 'year': '2019', 'director': 'Greta Gerwig'

In [11]:
retriever.retrieve("movies directed by Greta Gerwig with a rating above 9")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.indices.managed.vectara.retriever:Using query str: 
Using query str: 
INFO:llama_index.indices.managed.vectara.retriever:Using implicit filters: [('director', '==', 'Greta Gerwig'), ('rating', '>', 9)]
Using implicit filters: [('director', '==', 'Greta Gerwig'), ('rating', '>', 9)]


[NodeWithScore(node=TextNode(id_='19ce35022e05e1d8672d1c609ce71b7b0958e1ecfc61717f54d3df26bf43b2392dc12b4148842e582d50ba813aa9381141e9ddefd4f6315da36e85efb77844f5', embedding=None, metadata={'lang': 'eng', 'offset': '0', 'len': '79', 'year': '2023', 'director': 'Greta Gerwig', 'genre': 'fantasi', 'rating': '9.5'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='5d2839ded8712eeae0821b73d7eefedc5d3a3920bcd08073cddec1f13edfe429', text='Barbie suffers a crisis that leads her to question her world and her existence.', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.6245109)]

We can also include standard `VectaraRetriever` arguments in the `VectaraAutoRetriever`. For example, if we want to include a `filter` that would be added to any additional filtering from the query itself, we can do it as follows:

In [12]:
retriever = VectaraAutoRetriever(
    index, vector_store_info=vector_store_info, service_context=service_context,
    filter="doc.rating > 9"
)

In [13]:
retriever.retrieve("movies directed by Greta Gerwig")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.indices.managed.vectara.retriever:Using query str: movies directed by Greta Gerwig
Using query str: movies directed by Greta Gerwig
INFO:llama_index.indices.managed.vectara.retriever:Using implicit filters: [('director', '==', 'Greta Gerwig')]
Using implicit filters: [('director', '==', 'Greta Gerwig')]


[NodeWithScore(node=TextNode(id_='19ce35022e05e1d8672d1c609ce71b7b0958e1ecfc61717f54d3df26bf43b2392dc12b4148842e582d50ba813aa9381141e9ddefd4f6315da36e85efb77844f5', embedding=None, metadata={'lang': 'eng', 'offset': '0', 'len': '79', 'year': '2023', 'director': 'Greta Gerwig', 'genre': 'fantasi', 'rating': '9.5'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='5d2839ded8712eeae0821b73d7eefedc5d3a3920bcd08073cddec1f13edfe429', text='Barbie suffers a crisis that leads her to question her world and her existence.', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.60201997)]